# Merging Dataframes
-----------

- We used the DataFrame of store purchases from one of our previous lectures, where the index is a list of stores and the columns store purchase data. 

In [ ]:
import pandas as pd

df = pd.DataFrame([{'Name': 'Chris', 'Item Purchased': 'Sponge', 'Cost': 22.50},
                  {'Name': 'Kevyn', 'Item Purchased': 'Kitty Litter', 'Cost': 2.50},
                  {'Name': 'Filip', 'Item Purchased': 'Spoon', 'Cost': 5.00}],
                 index=['Store 1', 'Store 1', 'Store 2'])
df

- f we want to add some new column called Date to the DataFrame, we just use the square bracket operator directly on the DataFrame, as long as the column is as long as the rest of the records. 

In [ ]:
df['Date'] = ['December 1', 'January 1', 'mid-May']
df

- If we want to add some new field, may be a delivery flag, that's easy too since it's a scalar value. 

In [ ]:
df['Delivered'] = True
df

- The problem comes in when we have only a few items to add. In order for this to work, we have to supply pandas the list which is long enough for the DataFrame, so that each row could be populated. This means that we have to input none values ourselves.
- If each of our rows has a unique index, then we could just assign the new column identifier to the series. 

In [ ]:
df['Feedback'] = ['Positive', None, 'Negative']
df

- For instance, if we reset the index in this example so the DataFrame is labeled from 1 through 2, then we create a new series with these labels, we can apply it. 
- The nice aspect of this approach is that we could just ignore the items that we don't know about, and pandas will put missing values in for us. 

In [ ]:
adf = df.reset_index()
adf['Date'] = pd.Series({0: 'December 1', 2: 'mid-May'})
adf

- Now more commonly, we want to join two larger DataFrames together, and this is a bit more complex. 
- A Venn Diagram is traditionally used to show set membership. For example, the circle on the left is the population of students at a university. The circle on the right is the population of staff at a university. And the overlapping region in the middle are all of those students who are also staff. 
- We could think of these two populations as indices in separate DataFrames, maybe with the label of Person Name.

$~$

- When we want to join the DataFrames together, we have some choices to make.
 - First what if we want a list of all the people regardless of whether they're staff or student, and all of the information we can get on them? In database terminology, this is called a **full outer join** And in set theory, it's called a **union**. In the Venn diagram, it represents everyone in any circle. 
 - It's quite possible though that we only want those people who we have maximum information for, those people who are both staff and students. In database terminology, this is called an **inner join**. Or in set theory, the **intersection**. And this is represented in the Venn diagram as the overlapping parts of each circle. 
 
$~$

- An exapmle:

In [ ]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR'},
                         {'Name': 'Sally', 'Role': 'Course liasion'},
                         {'Name': 'James', 'Role': 'Grader'}])
staff_df = staff_df.set_index('Name')
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business'},
                           {'Name': 'Mike', 'School': 'Law'},
                           {'Name': 'Sally', 'School': 'Engineering'}])
student_df = student_df.set_index('Name')
print(staff_df.head())
print()
print(student_df.head())

- If we want the union of these, we would call merge passing in the DataFrame on the left and the DataFrame on the right, and telling merge that we want it to use an **outer** join. 
- We tell merge that we want to use the left and right indices as the joining columns. 
- We can see everyone is listed in this new dataframe, since Mike does not have a role, and John does not have a school, those cells are listed as missing values. 

In [ ]:
pd.merge(staff_df, student_df, how='outer', left_index=True, right_index=True)

-  If we wanted to get the intersection, that is, just those students who are also staff, we could set the how attribute to **inner**. And we set the resulting DataFrame has only James and Sally in it. 

In [ ]:
pd.merge(staff_df, student_df, how='inner', left_index=True, right_index=True)

- Now there are two other common use cases when merging DataFrames. Both are examples of what we would call set addition. 
 - The first is when we would want to get a list of all staff regardless of whether they were students or not. But if they were students, we would want to get their student details as well. To do this we would use a **left** join. 

In [ ]:
pd.merge(staff_df, student_df, how='left', left_index=True, right_index=True)

- 
 - Next, We want a list of all of the students and their roles if they were also staff. To do this we would do a **right** join. 

In [ ]:
pd.merge(staff_df, student_df, how='right', left_index=True, right_index=True)

- The merge method has a couple of other interesting parameters. 
 - First, you don't need to use indices to join on, you can use columns as well. Here's an example:

In [ ]:
staff_df = staff_df.reset_index()
student_df = student_df.reset_index()
pd.merge(staff_df, student_df, how='left', left_on='Name', right_on='Name')

- So what happens when we have conflicts between the DataFrames? Let's take a look by creating new staff and student DataFrames that have a location information added to them. In the staff DataFrame, this is an office location where we can find the staff person. And we can see the Director of HR is on State Street, while the two students are on Washington Avenue. But for the student DataFrame, the location information is actually their home address. 


In [ ]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR', 'Location': 'State Street'},
                         {'Name': 'Sally', 'Role': 'Course liasion', 'Location': 'Washington Avenue'},
                         {'Name': 'James', 'Role': 'Grader', 'Location': 'Washington Avenue'}])
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business', 'Location': '1024 Billiard Avenue'},
                           {'Name': 'Mike', 'School': 'Law', 'Location': 'Fraternity House #22'},
                           {'Name': 'Sally', 'School': 'Engineering', 'Location': '512 Wilson Crescent'}])
pd.merge(staff_df, student_df, how='left', left_on='Name', right_on='Name')

- The merge function preserves this information, but appends an `_x` or `_y` to help differentiate between which index went with which column of data. The `_x` is always the left DataFrame information, and the `_y` is always the right DataFrame information. you could control the names of `_x` and `_y` with additional parameters if you want to.

- let's talk about multi-indexing and multiple columns. 
- It's quite possible that the first name for students and staff might overlap, but the last name might not. In this case, we use a list of the multiple columns that should be used to join keys on the `left_on` and `right_on` parameters. 

In [ ]:
staff_df = pd.DataFrame([{'First Name': 'Kelly', 'Last Name': 'Desjardins', 'Role': 'Director of HR'},
                         {'First Name': 'Sally', 'Last Name': 'Brooks', 'Role': 'Course liasion'},
                         {'First Name': 'James', 'Last Name': 'Wilde', 'Role': 'Grader'}])
student_df = pd.DataFrame([{'First Name': 'James', 'Last Name': 'Hammond', 'School': 'Business'},
                           {'First Name': 'Mike', 'Last Name': 'Smith', 'School': 'Law'},
                           {'First Name': 'Sally', 'Last Name': 'Brooks', 'School': 'Engineering'}])
print(staff_df)
print()
print(student_df)
pd.merge(staff_df, student_df, how='inner', left_on=['First Name','Last Name'], right_on=['First Name','Last Name'])

-------
# Idiomatic Pandas: Making Code Pandorable
-------
- The best Python solutions to problems are celebrated as Idiomatic Python.
- An idiomatic solution is often one which has both high performance and high readability. 
- let's see a couple of key features of how you can make your code pandorable. 

- The first of these is called method chaining. 
- chain indexing:
 - `df.loc['Washtenaw']['Total Population']`
 - This is generally a bad practice, because pandas could return a copy of a view depending upon numpy.
 - code smell: If you see a `][` you should think carefully about what you are doing!

- Method chaining though, is a little bit different. The general idea behind method chaining is that every method on an object returns a reference to that object. The beauty of this is that you can condense many different operations on a DataFrame, for instance, into one line or at least one statement of code. 

In [ ]:
import pandas as pd
df = pd.read_csv('census.csv')
df

- Here's an example of two pieces of code in pandas using our census data. The first is the pandorable way to write the code with method chaining. 
- you can see that when we first run a `where` query, then a `dropna`, then a `set_index`, and then a `rename`. 

In [ ]:
(df.where(df['SUMLEV']==50)
    .dropna()
    .set_index(['STNAME','CTYNAME'])
    .rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'}))

- The second example is a more traditional way of writing code.

In [ ]:
df = df[df['SUMLEV']==50]
df.set_index(['STNAME','CTYNAME'], inplace=True)
df.rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'})

- Here's another pandas idiom. Python has a wonderful function called map, which is sort of a basis for functional programming in the language. When you want to use map in Python, you pass it some function you want called, and some iterable, like a list, that you want the function to be applied to. The results are that the function is called against each item in the list, and there's a resulting list of all of the evaluations of that function. 
- Pandas has a similar function called applymap. In applymap, you provide some function which should operate on each cell on a DataFrame, and the return set is itself a DataFrame. 
- Now I think applymap is fine, but I actually rarely use it. Instead, I find myself often wanting to map across all of the rows in a DataFrame. And pandas has a function that I use heavily there, called apply. ( Dr. Brook said! :D )

- First, we need to write a function which takes in a particular row of data, finds a minimum and maximum values, and returns a new row of data. 

In [ ]:
import numpy as np
def min_max(row):
    data = row[['POPESTIMATE2010',
                'POPESTIMATE2011',
                'POPESTIMATE2012',
                'POPESTIMATE2013',
                'POPESTIMATE2014',
                'POPESTIMATE2015']]
    return pd.Series({'min': np.min(data), 'max': np.max(data)})

- Then we just need to call `apply` on the DataFrame. `apply` takes the function and the axis on which to operate as parameters. Now, we have to be a bit careful, we've talked about axis zero being the rows of the DataFrame in the past. But this parameter is really the **parameter of the index to use**. 

In [ ]:
df.apply(min_max, axis=1)

- If you're doing this as part of data cleaning your likely to find yourself wanting to add new data to the existing DataFrame. In that case you just take the row values and add in new columns indicating the max and minimum scores. 

In [ ]:
import numpy as np
def min_max(row):
    data = row[['POPESTIMATE2010',
                'POPESTIMATE2011',
                'POPESTIMATE2012',
                'POPESTIMATE2013',
                'POPESTIMATE2014',
                'POPESTIMATE2015']]
    row['max'] = np.max(data)
    row['min'] = np.min(data)
    return row
df.apply(min_max, axis=1)

In [ ]:
rows = ['POPESTIMATE2010',
        'POPESTIMATE2011',
        'POPESTIMATE2012',
        'POPESTIMATE2013',
        'POPESTIMATE2014',
        'POPESTIMATE2015']
df.apply(lambda x: np.max(x[rows]), axis=1)

-------
# Group by
-------

- We've seen that even though PANDAS allows us to iterate over every row in a data frame this is generally a slow way to accomplish a given task and it's not very pandorable. 
- For instance, if we wanted to write some code to iterate over all the of the states and generate a list of the average census population numbers. We could do so using a loop in the `unique` function.
- Another option is to use the dataframe `groupby` function. This function takes some column name or names and splits the dataframe up into chunks based on those names, it returns a dataframe group by object. Which can be iterated upon, and then returns a tuple where the first item is the group condition, and the second item is the data frame reduced by that grouping. 

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('census.csv')
df = df[df['SUMLEV']==50]
df

In [ ]:
%%timeit -n 10
for state in df['STNAME'].unique():
    avg = np.average(df.where(df['STNAME']==state).dropna()['CENSUS2010POP'])
    print('Counties in state ' + state + ' have an average population of ' + str(avg))

In [ ]:
%%timeit -n 10
for group, col in df.groupby('STNAME'):
    avg = np.average(col['CENSUS2010POP'])
    print('Counties in state ' + group + ' have an average population of ' + str(avg))

- you can actually provide a function to group by as well and use that to segment your data. This is a bit of a fabricated example but lets say that you have a big batch job with lots of processing and you want to work on only a third or so of the states at a given time. 
- We could create some function which returns a number between zero and two based on the first character of the state name. Then we can tell group by to use this function to split up our data frame. It's important to note that in order to do this you need to set the index of the data frame to be the column that you want to group by first. 

In [ ]:
df.head()

- Here's an example. We'll create some new function called `fun` and if the first letter of the parameter is a capital `M` we'll return a 0. If it's a capital `Q` we'll return a 1 and otherwise we'll return a 2. 
- Then we'll pass this function to the data frame `groupby`, and see that the data frame is segmented by the calculated group number.

$~$

- This kind of technique, which is sort of a light weight hashing, is commonly used to distribute tasks across multiple workers or cores in a processor, nodes in a supercomputer, or disks in a database.


In [ ]:
df = df.set_index('STNAME')

def fun(item):
    if item[0]<'M':
        return 0
    if item[0]<'Q':
        return 1
    return 2

for group, frame in df.groupby(fun):
    print('There are ' + str(len(frame)) + ' records in group ' + str(group) + ' for processing.')


- A common work flow with `groupby` is that you split your data, you apply some function, then you combine the results. This is called **split apply combine** pattern.
- we've seen the splitting method, but what about apply? Certainly iterative methods as we've seen can do this, but the `groupby` object also has a method called `agg` which is short for aggregate. This method applies a function to the column or columns of data in the group, and returns the results. 
- With `agg`, you simply pass in a dictionary of the column names that you're interested in, and the function that you want to apply.
- For instance to build a summary data frame for the average populations per state, we could just give `agg` a dictionary with the Census 2010 pop key and the numpy average function

In [ ]:
df = pd.read_csv('census.csv')
df = df[df['SUMLEV']==50]

In [ ]:
df.groupby('STNAME').agg({'CENSUS2010POP': np.average})

- You see, when you pass in a dictionary to `agg`, it can be used to either to identify the columns to apply a function on or to name an output column if there's multiple functions to be run. The difference depends on the keys that you pass in from the dictionary and how they're named. 
- In short, while much of the documentation and examples will talk about a single groupby object, there's really two different objects. The data frame groupby and the series groupby. And these objects behave a little bit differently with aggregate. 

In [ ]:
print(type(df.groupby(level=0)['POPESTIMATE2010','POPESTIMATE2011']))
print(type(df.groupby(level=0)['POPESTIMATE2010']))

In [ ]:
df.set_index('STNAME').groupby(level=0)['CENSUS2010POP'].agg([np.average, np.sum])

- We can do the same thing with a data frame instead of a series. We set the index to be the state name, we group by the index, and we project two columns. The population estimate in 2010, the population estimate in 2011. 
- When we call aggregate with two parameters, it builds a nice hierarchical column space and all of our functions are applied. 

In [ ]:
(df.set_index('STNAME').groupby(level=0)['POPESTIMATE2010','POPESTIMATE2011']
    .agg([np.average, np.sum]))